In [11]:
import pandas as pd
import numpy as np
import os
from datasets import load_dataset

In [18]:
llama = pd.read_csv('C:/Users/sethh/Documents/ariel/comp_ling_proj/results/large_processed.csv')
print(llama.shape)
llama.head()

(13139, 3)


,prompt,original_response,llama_response
0,The following is a conversation with a teacher...,"Ah OK, so good practice then...is that an IELT...",I can't imagine that this is your first time t...
1,The following is a conversation with a teacher...,I've lost track of how many you've done,. Do you think you'll do well this time?. Stud...
2,The following is a conversation with a teacher...,Do you mean get a score you get in practice te...,Or a score you get in real exams?. Student: I'...
3,The following is a conversation with a teacher...,Wow that's really good - well done...you've de...,Do you think you'll manage to get one hundred%...
4,The following is a conversation with a teacher...,OK that's quite common to be better at reading...,Do you have any idea what you might be able to...


In [21]:
ds = load_dataset("HuggingFaceH4/ultrafeedback_binarized", split="train_prefs")
hf = pd.DataFrame(ds)
hf.head()

,prompt,prompt_id,chosen,rejected,messages,score_chosen,score_rejected
0,how can i develop a habit of drawing daily,086b3e24f29b8956a01059f79c56db35d118a06fb6b844...,[{'content': 'how can i develop a habit of dra...,[{'content': 'how can i develop a habit of dra...,[{'content': 'how can i develop a habit of dra...,8.5,8.5
1,how can I transform the getPosition method of ...,2766cbd1fed7f982d94b031596e771c841668bd8913839...,[{'content': 'how can I transform the getPosit...,[{'content': 'how can I transform the getPosit...,[{'content': 'how can I transform the getPosit...,6.5,6.5
2,"Given a sentence in French, provide an equival...",0efb42706b3fcc906f579505c7cc0c4e68a640ab3862b1...,"[{'content': 'Given a sentence in French, prov...","[{'content': 'Given a sentence in French, prov...","[{'content': 'Given a sentence in French, prov...",6.0,3.0
3,"Which animal has two hands, a hyrax or a dog?",d169f4610d69b94f54b6923e11aeffe2e321e272395686...,"[{'content': 'Which animal has two hands, a hy...","[{'content': 'Which animal has two hands, a hy...","[{'content': 'Which animal has two hands, a hy...",9.0,8.0
4,Can you explain more about how Tamping Coke is...,ac205b0f69d45eae8dbf446673bd78284e230b6ce3381e...,[{'content': 'Can you explain more about how T...,[{'content': 'Can you explain more about how T...,[{'content': 'Can you explain more about how T...,8.0,6.0


In [19]:
llama=llama.rename(columns={"prompt": "prompt", "original_response": "chosen", "llama_response": "rejected"})
llama.head()

,prompt,chosen,rejected
0,The following is a conversation with a teacher...,"Ah OK, so good practice then...is that an IELT...",I can't imagine that this is your first time t...
1,The following is a conversation with a teacher...,I've lost track of how many you've done,. Do you think you'll do well this time?. Stud...
2,The following is a conversation with a teacher...,Do you mean get a score you get in practice te...,Or a score you get in real exams?. Student: I'...
3,The following is a conversation with a teacher...,Wow that's really good - well done...you've de...,Do you think you'll manage to get one hundred%...
4,The following is a conversation with a teacher...,OK that's quite common to be better at reading...,Do you have any idea what you might be able to...


In [22]:
hf = hf.drop(['prompt_id', 'messages', "score_chosen", "score_rejected"], axis=1)
hf.head()

,prompt,chosen,rejected
0,how can i develop a habit of drawing daily,[{'content': 'how can i develop a habit of dra...,[{'content': 'how can i develop a habit of dra...
1,how can I transform the getPosition method of ...,[{'content': 'how can I transform the getPosit...,[{'content': 'how can I transform the getPosit...
2,"Given a sentence in French, provide an equival...","[{'content': 'Given a sentence in French, prov...","[{'content': 'Given a sentence in French, prov..."
3,"Which animal has two hands, a hyrax or a dog?","[{'content': 'Which animal has two hands, a hy...","[{'content': 'Which animal has two hands, a hy..."
4,Can you explain more about how Tamping Coke is...,[{'content': 'Can you explain more about how T...,[{'content': 'Can you explain more about how T...


In [27]:
# Combine datasets (75% uploaded, 25% HuggingFace)
hf_sample = hf.sample(n=3285, random_state=42)
merged = pd.concat([llama, hf_sample])
shuffled = merged.sample(frac=1, random_state=42).reset_index(drop=True)
shuffled.to_csv("combined_dataset.csv", index=False)

In [29]:
df = pd.read_csv('combined_dataset.csv')
import json

In [31]:
import pandas as pd
import json

# Load the CSV file
df = pd.read_csv('combined_dataset.csv')

# Define a function to convert the DataFrame to the desired JSON format
def convert_to_dpo_format(df):
    dpo_list = []
    for _, row in df.iterrows():
        dpo_entry = {
            "instruction": row['prompt'],
            "input": "",  # Assuming input is optional and not provided
            "chosen": row['chosen'],
            "rejected": row['rejected']
        }
        dpo_list.append(dpo_entry)
    return dpo_list

# Convert the DataFrame to the DPO format
dpo_data = convert_to_dpo_format(df)

# Output the data as JSON
with open('dpo_dataset.json', 'w') as f:
    json.dump(dpo_data, f, indent=2)

print("Conversion complete. DPO dataset saved as 'dpo_dataset.json'.")

Conversion complete. DPO dataset saved as 'dpo_dataset.json'.
